<a href="https://colab.research.google.com/github/yyc0314/Fooood-map/blob/main/crawler/dynamic_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""dynamic.ipynb

Original file is located at
    https://colab.research.google.com/drive/1RbSbNSCpOPa2yzyVc-vLG1sjiHLgEmQk

### 透過指定路徑設定Webdriver
* download driver from <a href='https://chromedriver.storage.googleapis.com/index.html'>here</a>
"""

import

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import json
from bs4 import BeautifulSoup
import numpy as np
import time
import re
import pandas as pd
import csv

path

In [ ]:
r_60_path=""
r_60=pd.read_csv(r_60_path)
#print(r_60)
href=list(r_60["href"])

r_60_name=list(r_60["restaurant_name"])
for i in r_60_name:
    print(i)

特殊字元

In [ ]:
emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # 表情符號
        u"\U0001F300-\U0001F5FF"  # 圖案
        u"\U0001F680-\U0001F6FF"  # 運輸和地圖
        u"\U0001F1E0-\U0001F1FF"  # 國旗
                           "]+", flags=re.UNICODE)

def rer(sen):
    sen=emoji_pattern.sub(r'', sen)
    sen=re.sub(r'[，、丶!！?？~～。＝＋－\\n-]', '|', sen)
    '''sen=re.sub(",","|",sen)
    sen=re.sub("，","|",sen)
    sen=re.sub("、","|",sen)
    sen=re.sub("丶","|",sen)
    sen=re.sub("！","|",sen)
    sen=re.sub("？","|",sen)
    sen=re.sub("!","|",sen)
    sen=re.sub("\?","|",sen)
    sen=re.sub("。","|",sen)
    sen=re.sub("～","|",sen)
    sen=re.sub("~","|",sen)
    sen=re.sub("\\n","",sen)'''
    return sen

抓餐廳

In [ ]:
# Set up the webdriver with Service object
service = Service('/path/to/chromedriver')
driver = webdriver.Chrome(service=service)

# Navigate to Google Maps
KEYWORD = '高雄市區餐廳'
driver.get(f"https://www.google.com/maps/search/{KEYWORD}")

restaurants = {}

#持續捲動頁面，直到餐廳個數夠
while len(restaurants)<20:
    search_result = driver.find_elements(By.CSS_SELECTOR, '[role="article"]>a')
    for item in search_result:
        if len(restaurants) >= 20:
            break

        restaurant_name = item.get_attribute('aria-label')
        restaurant_url = item.get_attribute('href')

        if restaurant_url not in href:
            restaurants[restaurant_name] = {}
            restaurants[restaurant_name]["href"]=restaurant_url
            restaurants[restaurant_name]["star"]=[]
            restaurants[restaurant_name]["tag"]=[]
            restaurants[restaurant_name]["comment"]=[]

            print(restaurant_name,restaurant_url,len(restaurants))

    #print(restaurants)

    if len(restaurants)<20:
        time.sleep(2)
        driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.CLASS_NAME, 'qjESne'))
print(len(restaurants))

path="" #\\rs_10.csv"

In [ ]:
# CSV欄位
fieldnames = ["restaurant_name", "href", "tag", "comment", "star"]

with open("rs_100.csv", mode='a',encoding="utf-8", newline="") as csv_file:
    writer = csv.DictWriter(csv_file,fieldnames=fieldnames)
    #writer.writeheader() # 寫入欄位

    for restaurant_name in restaurants:
        driver.get(f"{restaurants[restaurant_name]['href']}")
        #點評論->到總評論
        driver.find_element(By.XPATH,"//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[3]/div/div/button[2]").click()
        i=0
        #評論下滑
        while True:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", driver.find_element(By.CLASS_NAME, 'qjESne'))
                time.sleep(5)
                i+=1
                if i%10==0:
                    print(restaurant_name,i)
                j=0
                reviews = driver.find_elements(By.CLASS_NAME, 'MyEned')
                for review in reviews:
                    # 展開評論全文
                    more_btn = review.find_elements(By.CSS_SELECTOR, 'span:nth-child(2) > button')
                    if j%10==0:
                        print("open",j)
                    j+=1

                    if len(more_btn) > 0:
                        more_btn[0].click()
                        time.sleep(1)

                if driver.execute_script("return window.innerHeight+window.pageYOffset") > driver.execute_script("return document.body.scrollHeight"):
                    break
                if i>100:
                    break
            except:
                break

        print(restaurant_name,"buttom")
        reviews = driver.find_elements(By.CLASS_NAME, 'MyEned')

        i=0
        '''for review in reviews:
            # 展開評論全文
            more_btn = review.find_elements(By.CSS_SELECTOR, 'span:nth-child(2) > button')
            print(i)
            i+=1
            if len(more_btn) > 0:
                more_btn[0].click()
                time.sleep(1)'''
                #print("|-|")
        soup=BeautifulSoup(driver.page_source, 'html.parser')

        #抓評分
        star = driver.find_element(By.CSS_SELECTOR, 'div.jANrlb > div.fontDisplayLarge')
        restaurants[restaurant_name]["star"].append(float(star.text))
        #restaurants[restaurant_name]["href"]=restaurant_url
        #print(star.text,"kkk")

        #抓評論+資料清理
        comments = soup.find_all('div', {'class':  'MyEned'})
        for cm in comments:
            sen=cm.text
            comment=""
            sen=rer(sen)
            comment+=sen
            #print(comment)
            restaurants[restaurant_name]["comment"].append(comment)
            #print(comment.text)

        #抓tag
        tags = driver.find_elements(By.CSS_SELECTOR, 'button > span > span.uEubGf.fontBodyMedium')
        i=0
        for tag in tags:
            if tag.text!="" and i!=0:
                restaurants[restaurant_name]["tag"].append((tag.text))
                #print(tag.text)
            i+=1

        print(restaurant_name,len(restaurants[restaurant_name]["comment"]))

        writer.writerow({
                "restaurant_name": restaurant_name,
                "href": restaurants[restaurant_name]["href"],
                "tag": restaurants[restaurant_name]["tag"],
                "comment": restaurants[restaurant_name]["comment"],
                "star": restaurants[restaurant_name]["star"]
            })

In [ ]:
for r in restaurants:
    print(r)
    print(restaurants[r]["href"])
    print(restaurants[r]["star"])
    print(len(restaurants[r]["comment"]))
    print(restaurants[r]["tag"])